In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_json('data_106.json')

In [3]:
df.columns

Index(['raw', 'from', 'to', 'path', 'via', 'messagecapable', 'format',
       'posambiguity', 'symbol', 'symbol_table', 'latitude', 'longitude',
       'comment', 'mbits', 'mtype', 'speed', 'course', 'altitude',
       'object_name', 'alive', 'raw_timestamp', 'timestamp', 'object_format',
       'addresse', 'tUNIT', 'phg', 'phg_power', 'phg_height', 'phg_gain',
       'phg_dir', 'phg_range', 'rng', 'daodatumbyte', 'message_text', 'msgNo',
       'weather', 'status', 'text', 'response', 'gpsfixstatus', 'tPARM',
       'tEQNS', 'tBITS', 'title', 'bid', 'identifier', 'phg_rate',
       'wx_raw_timestamp', 'telemetry', 'radiorange', 'id', 'type', 'body',
       'ackMsgNo'],
      dtype='object')

In [4]:
column = "latitude"
df.loc[~df[column].isna(), column]

0       41.442500
1      -34.418667
2       45.307500
3       27.058333
5       41.005000
          ...    
9238    41.941833
9239    35.657333
9241    40.089667
9242    43.108667
9246    53.201833
Name: latitude, Length: 7348, dtype: float64

In [10]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude")

fig.update_layout(mapbox_style = 'carto-positron')
fig.show()